In [24]:
import pandas as pd
from openpyxl.workbook import Workbook


In [25]:
dataframe1 = pd.read_excel('AlzheimerData.xlsx')

In [26]:
dataframe1

,Target,"Descriptors or Properties or features, or attributes",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
1,517.06,9.233,1.13,0,7,16,4,0.067,4,0,12,2,4,39.3,3.6,7.1
2,504.06,9.001,1.129,0,8,16,4,0.068,4,0,12,1,4,39.3,1.8,7.1
3,490.03,9.246,1.128,0,8,16,3,0.054,4,0,12,1,4,39.6,1.9,7.5
4,1443.08,6.414,1.146,0,41,20,42,0.187,14,#@@^,6,13,17,32,5.8,7.6
5,517.06,9.233,1.13,0,7,16,4,0.067,4,0,12,2,4,39.3,3.6,7.1
6,514.57,8.3,1.127,0,7,23,5,0.076,5,0,18,4,3,46.8,6.5,4.8
7,518.09,8.781,1.129,0,8,16,5,0.081,4,0,12,1,4,39,1.7,6.8
8,531.09,9.002,1.13,0,7,16,4,0.063,4,0,#@@^,2,4,39,3.4,6.8
9,516.59,8.072,1.129,0,7,22,5,0.074,4,0,18,4,3,45.3,6.3,4.7


In [27]:
dataframe1.columns = dataframe1.loc[0,:]

In [28]:
dataframe1.columns

Index(['Y ', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',
       'X11', 'X12', 'X13', 'X14', 'X15'],
      dtype='object', name=0)

In [29]:
dataframe1 = dataframe1.loc[1:,:]

In [30]:
dataframe1 

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
1,517.06,9.233,1.13,0,7,16,4,0.067,4,0,12,2,4,39.3,3.6,7.1
2,504.06,9.001,1.129,0,8,16,4,0.068,4,0,12,1,4,39.3,1.8,7.1
3,490.03,9.246,1.128,0,8,16,3,0.054,4,0,12,1,4,39.6,1.9,7.5
4,1443.08,6.414,1.146,0,41,20,42,0.187,14,#@@^,6,13,17,32,5.8,7.6
5,517.06,9.233,1.13,0,7,16,4,0.067,4,0,12,2,4,39.3,3.6,7.1
6,514.57,8.3,1.127,0,7,23,5,0.076,5,0,18,4,3,46.8,6.5,4.8
7,518.09,8.781,1.129,0,8,16,5,0.081,4,0,12,1,4,39,1.7,6.8
8,531.09,9.002,1.13,0,7,16,4,0.063,4,0,#@@^,2,4,39,3.4,6.8
9,516.59,8.072,1.129,0,7,22,5,0.074,4,0,18,4,3,45.3,6.3,4.7
10,505.05,9.183,1.132,0,8,16,4,0.069,4,0,12,2,4,38.2,3.6,7.3


In [31]:
df_clean = dataframe1.copy()

In [32]:
df_clean = df_clean.loc[:, (df_clean != 0).any(axis=0)]

In [33]:
df_clean

,Y,X1,X2,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
1,517.06,9.233,1.13,7,16,4,0.067,4,0,12,2,4,39.3,3.6,7.1
2,504.06,9.001,1.129,8,16,4,0.068,4,0,12,1,4,39.3,1.8,7.1
3,490.03,9.246,1.128,8,16,3,0.054,4,0,12,1,4,39.6,1.9,7.5
4,1443.08,6.414,1.146,41,20,42,0.187,14,#@@^,6,13,17,32,5.8,7.6
5,517.06,9.233,1.13,7,16,4,0.067,4,0,12,2,4,39.3,3.6,7.1
6,514.57,8.3,1.127,7,23,5,0.076,5,0,18,4,3,46.8,6.5,4.8
7,518.09,8.781,1.129,8,16,5,0.081,4,0,12,1,4,39,1.7,6.8
8,531.09,9.002,1.13,7,16,4,0.063,4,0,#@@^,2,4,39,3.4,6.8
9,516.59,8.072,1.129,7,22,5,0.074,4,0,18,4,3,45.3,6.3,4.7
10,505.05,9.183,1.132,8,16,4,0.069,4,0,12,2,4,38.2,3.6,7.3


In [40]:
keep=df_clean.astype('str').replace(to_replace="^[^0-9]+$", value='', regex=True).applymap(len).max()<5

In [45]:
keep

0
Y      False
X1     False
X2     False
X4      True
X5      True
X6      True
X7     False
X8      True
X9      True
X10     True
X11     True
X12     True
X13     True
X14     True
X15     True
dtype: bool

In [43]:
df_clean.loc[:,keep].replace(to_replace="^[^0-9]+$", value=0, regex=True, inplace=True)

C:\Users\azadeh\anaconda3\lib\site-packages\pandas\core\frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [44]:
df_clean

,Y,X1,X2,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
1,517.06,9.233,1.13,7,16,4,0.067,4,0,12,2,4,39.3,3.6,7.1
2,504.06,9.001,1.129,8,16,4,0.068,4,0,12,1,4,39.3,1.8,7.1
3,490.03,9.246,1.128,8,16,3,0.054,4,0,12,1,4,39.6,1.9,7.5
4,1443.08,6.414,1.146,41,20,42,0.187,14,#@@^,6,13,17,32,5.8,7.6
5,517.06,9.233,1.13,7,16,4,0.067,4,0,12,2,4,39.3,3.6,7.1
6,514.57,8.3,1.127,7,23,5,0.076,5,0,18,4,3,46.8,6.5,4.8
7,518.09,8.781,1.129,8,16,5,0.081,4,0,12,1,4,39,1.7,6.8
8,531.09,9.002,1.13,7,16,4,0.063,4,0,#@@^,2,4,39,3.4,6.8
9,516.59,8.072,1.129,7,22,5,0.074,4,0,18,4,3,45.3,6.3,4.7
10,505.05,9.183,1.132,8,16,4,0.069,4,0,12,2,4,38.2,3.6,7.3


In [20]:
str_cols=df_clean.dtypes=='object'
df2=df_clean.loc[: , str_cols]
keep=df2.astype('str').replace(to_replace=r'[0-9]', value='', regex=True).applymap(len).max()<5
df2=df2.loc[:,keep].replace(to_replace=r'^[0-9]', value='0', regex=True)
df=pd.concat([df_clean[~str_cols],df2],1)

C:\Users\azadeh\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [12]:
df_clean.replace(to_replace ="^[^0-9]+$", value =0, regex=True, inplace=True)

In [13]:
df_clean

,Y,X1,X2,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
1,517.06,9.233,1.130,7,16,4,0.067,4,0,12.0,2,4,39.3,3.6,7.1
2,504.06,9.001,1.129,8,16,4,0.068,4,0,12.0,1,4,39.3,1.8,7.1
3,490.03,9.246,1.128,8,16,3,0.054,4,0,12.0,1,4,39.6,1.9,7.5
4,1443.08,6.414,1.146,41,20,42,0.187,14,0,6.0,13,17,32.0,5.8,7.6
5,517.06,9.233,1.130,7,16,4,0.067,4,0,12.0,2,4,39.3,3.6,7.1
6,514.57,8.300,1.127,7,23,5,0.076,5,0,18.0,4,3,46.8,6.5,4.8
7,518.09,8.781,1.129,8,16,5,0.081,4,0,12.0,1,4,39.0,1.7,6.8
8,531.09,9.002,1.130,7,16,4,0.063,4,0,0.0,2,4,39.0,3.4,6.8
9,516.59,8.072,1.129,7,22,5,0.074,4,0,18.0,4,3,45.3,6.3,4.7
10,505.05,9.183,1.132,8,16,4,0.069,4,0,12.0,2,4,38.2,3.6,7.3


In [20]:
df_clean = df_clean.loc[:, (df_clean != 0).any(axis=0)]

In [21]:
df_clean

,Y,X1,X2,X4,X5,X6,X7,X8,X10,X11,X12,X13,X14,X15
1,517.06,9.233,1.130,7,16,4,0.067,4,12.0,2,4,39.3,3.6,7.1
2,504.06,9.001,1.129,8,16,4,0.068,4,12.0,1,4,39.3,1.8,7.1
3,490.03,9.246,1.128,8,16,3,0.054,4,12.0,1,4,39.6,1.9,7.5
4,1443.08,6.414,1.146,41,20,42,0.187,14,6.0,13,17,32.0,5.8,7.6
5,517.06,9.233,1.130,7,16,4,0.067,4,12.0,2,4,39.3,3.6,7.1
6,514.57,8.300,1.127,7,23,5,0.076,5,18.0,4,3,46.8,6.5,4.8
7,518.09,8.781,1.129,8,16,5,0.081,4,12.0,1,4,39.0,1.7,6.8
8,531.09,9.002,1.130,7,16,4,0.063,4,0.0,2,4,39.0,3.4,6.8
9,516.59,8.072,1.129,7,22,5,0.074,4,18.0,4,3,45.3,6.3,4.7
10,505.05,9.183,1.132,8,16,4,0.069,4,12.0,2,4,38.2,3.6,7.3


In [22]:
df_clean.stack().std()  

148.60477040881133

In [23]:
 df_clean.mean()

0
Y      549.642000
X1       8.184320
X2       1.131160
X4       8.440000
X5      16.480000
X6       6.240000
X7       0.080042
X8       4.240000
X10     12.666667
X11      2.680000
X12      4.200000
X13     40.624000
X14      3.992000
X15      6.064000
dtype: float64

In [18]:
x_standarization = (df_clean - df_clean.mean()) / (df_clean.stack().std())

In [19]:
x_standarization

,Y,X1,X2,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
1,-0.226231,0.007281,-0.000008,-0.009999,-0.003333,-0.015553,-0.000091,-0.001666,0.0,-0.004629,-0.004722,-0.001389,-0.009193,-0.002722,0.007193
2,-0.316496,0.005671,-0.000015,-0.003055,-0.003333,-0.015553,-0.000084,-0.001666,0.0,-0.004629,-0.011665,-0.001389,-0.009193,-0.015220,0.007193
3,-0.413912,0.007372,-0.000022,-0.003055,-0.003333,-0.022497,-0.000181,-0.001666,0.0,-0.004629,-0.011665,-0.001389,-0.007110,-0.014526,0.009971
4,6.203531,-0.012292,0.000103,0.226078,0.024441,0.248297,0.000743,0.067768,0.0,-0.046290,0.071656,0.088876,-0.059880,0.012554,0.010665
5,-0.226231,0.007281,-0.000008,-0.009999,-0.003333,-0.015553,-0.000091,-0.001666,0.0,-0.004629,-0.004722,-0.001389,-0.009193,-0.002722,0.007193
6,-0.243520,0.000803,-0.000029,-0.009999,0.045271,-0.008610,-0.000028,0.005277,0.0,0.037032,0.009165,-0.008332,0.042883,0.017414,-0.008777
7,-0.219079,0.004143,-0.000015,-0.003055,-0.003333,-0.008610,0.000007,-0.001666,0.0,-0.004629,-0.011665,-0.001389,-0.011276,-0.015914,0.005110
8,-0.128815,0.005678,-0.000008,-0.009999,-0.003333,-0.015553,-0.000118,-0.001666,0.0,-0.087950,-0.004722,-0.001389,-0.011276,-0.004111,0.005110
9,-0.229495,-0.000780,-0.000015,-0.009999,0.038328,-0.008610,-0.000042,-0.001666,0.0,0.037032,0.009165,-0.008332,0.032468,0.016025,-0.009471
10,-0.309622,0.006934,0.000006,-0.003055,-0.003333,-0.015553,-0.000077,-0.001666,0.0,-0.004629,-0.004722,-0.001389,-0.016831,-0.002722,0.008582


In [26]:
x_Normalization = (df_clean - df_clean.min()) / ((df_clean.max() - df_clean.min()))

In [27]:
x_Normalization

,Y,X1,X2,X4,X5,X6,X7,X8,X10,X11,X12,X13,X14,X15
1,0.040970,0.977761,0.279070,0.105263,0.695652,0.025641,0.097744,0.090909,0.666667,0.153846,0.133333,0.470968,0.537313,0.898734
2,0.027507,0.953193,0.255814,0.131579,0.695652,0.025641,0.105263,0.090909,0.666667,0.076923,0.133333,0.470968,0.268657,0.898734
3,0.012977,0.979138,0.232558,0.131579,0.695652,0.000000,0.000000,0.090909,0.666667,0.076923,0.133333,0.490323,0.283582,0.949367
4,1.000000,0.679233,0.651163,1.000000,0.869565,1.000000,1.000000,1.000000,0.333333,1.000000,1.000000,0.000000,0.865672,0.962025
5,0.040970,0.977761,0.279070,0.105263,0.695652,0.025641,0.097744,0.090909,0.666667,0.153846,0.133333,0.470968,0.537313,0.898734
6,0.038391,0.878958,0.209302,0.105263,1.000000,0.051282,0.165414,0.181818,1.000000,0.307692,0.066667,0.954839,0.970149,0.607595
7,0.042037,0.929895,0.255814,0.131579,0.695652,0.051282,0.203008,0.090909,0.666667,0.076923,0.133333,0.451613,0.253731,0.860759
8,0.055500,0.953299,0.279070,0.105263,0.695652,0.025641,0.067669,0.090909,0.000000,0.153846,0.133333,0.451613,0.507463,0.860759
9,0.040483,0.854813,0.255814,0.105263,0.956522,0.051282,0.150376,0.090909,1.000000,0.307692,0.066667,0.858065,0.940299,0.594937
10,0.028532,0.972466,0.325581,0.131579,0.695652,0.025641,0.112782,0.090909,0.666667,0.153846,0.133333,0.400000,0.537313,0.924051
